Compute a model's predicted EEG response to held-out data.
Pre-compute epoched forms of the response as well.
Writes to a shared hdf5 result file.

The outcome of `rollout` can/should be combined with an epoched plotting analysis for visual comparison of models.

In [1]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
from pathlib import Path
import sys

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.base import clone
import torch
from tqdm.auto import tqdm, trange

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.chdir("..")
!pwd

sys.path.append(str(Path(".").resolve()))
from berp.datasets import BerpDataset, NestedBerpDataset
from berp.models import load_model
import berp.models.reindexing_regression as rr
from berp.models.trf_em import GroupVanillaTRFForwardPipeline, GroupBerpFixedTRFForwardPipeline
import berp.trainer  # sets some nice yaml dump defaults
from berp.viz import make_word_onset_epochs, make_word_recognition_epochs, epoch_ts
from berp.viz.trf import trf_to_dataframe, plot_trf_coefficients

/net/vast-storage.ib.cluster/scratch/vast/cpl/jgauthie/scratch/bayesian-erp


In [4]:
# PARAMETERS

workflow = "heilbron2022"
lm = "EleutherAI/gpt-neo-2.7B/n10000"

# paradigm = "cannon_wide"
# run_name = "trf-berp-cannon_3quantiles.29720801"
paradigm = "cannon_wide"
run_name = "trf-cannon_3quantiles"
# paradigm = "wide"
# run_name = "trf"

epoch_kwargs = dict(
    tmin=-12/128, tmax=96/128,
    baseline=True,
    baseline_tmin=-12/128, baseline_tmax=0,
)

out_dir = "scripts/rollouts"
device = "cpu"

In [5]:
model_dir = f"workflow/{workflow}/results{'_' + paradigm if paradigm is not None else ''}/{lm}/{run_name}"

In [6]:
Path(out_dir).mkdir(exist_ok=True)
out_path = Path(out_dir) / f"{workflow}.{lm.replace('/', '--')}.h5"
store = pd.HDFStore(out_path)

## Load

In [7]:
trainers = berp.trainer.load_trainers_from_checkpoints([model_dir], device=device)

  0%|          | 0/1 [00:00<?, ?model/s]



===== workflow/heilbron2022/results_cannon_wide/EleutherAI/gpt-neo-2.7B/n10000/trf-cannon_3quantiles
confusion: tensor of type torch.float64, shape torch.Size([40, 40])
encoder: "TemporalReceptiveField(alpha=82468.40682196281, fit_intercept=True,\n  \
  \                     name='baf59f60a976403db080a7fc4a154857',\n               \
  \        optim=AdamSolver(n_batches=50,\n                                      \
  \  name='baf59f60a976403db080a7fc4a154857',\n                                  \
  \      random_state=RandomState(MT19937) at 0x2B70159C1440),\n                 \
  \      sfreq=128.0, tmax=0.75, tmin=0.0)"
encoder__alpha: 82468.40682196281
encoder__fit_intercept: true
encoder__init_scale: 0.1
encoder__n_outputs: null
encoder__name: baf59f60a976403db080a7fc4a154857
encoder__optim: "AdamSolver(n_batches=50, name='baf59f60a976403db080a7fc4a154857',\n\
  \           random_state=RandomState(MT19937) at 0x2B70159C1440)"
encoder__optim__batch_size: 512
encoder__optim__early_s

In [8]:
trainer = trainers[0]

In [9]:
pipe = trainer.model

In [10]:
# TODO allow different sensors within and across pipelines
all_sensors = set(tuple(encoder.output_names)
    for trainer in trainers
    for encoder in trainer.model.encoders_.values())
assert len(all_sensors) == 1

In [11]:
ordered_sensors = next(iter(trainers[0].model.encoders_.values())).output_names

## Store epoched data

In [12]:
store_ds = trainer.data_test

In [13]:
def prepare_all_epochs(ts=None, baseline=False, word_meta=None):
    """
    Prepare epochs dataframe with relevant metadata, possibly swapping
    in alternate time series data with `ts`.
    """
    # # DEV
    # ds_ = dict(list(ds.items())[:2])
    ds_ = store_ds

    kwargs = epoch_kwargs | {"baseline": baseline}
    epoch_dfs = {
        ds_i.name: make_word_onset_epochs(
            ds_i,
            ts=ts[ds_i.name] if ts is not None else None,
            **kwargs)
        for ds_i in tqdm(ds_.datasets)
    }
    epoch_df = pd.concat(epoch_dfs, names=["dataset", "epoch", "sample", "sensor_idx"]) \
        .set_index("word_idx", append=True) \
        .sort_index()

    if word_meta is not None:
        word_meta = pd.concat(word_meta, names=["dataset"]).droplevel(1) \
            .set_index("word_idx", append=True).sort_index()
        epoch_df = pd.merge(epoch_df, word_meta, left_index=True, right_index=True)
        
    epoch_df = epoch_df.reset_index()
    epoch_df["sensor"] = epoch_df.sensor_idx.map(dict(enumerate(ordered_sensors)))
    
    return epoch_df

In [14]:
if "/epochs/observed" not in store:
    store["epochs/observed"] = prepare_all_epochs()

In [15]:
pred_ts = [pipe.predict(ds_i) for ds_i in tqdm(store_ds.datasets, desc="Predicting")]

word_meta = None
if hasattr(pipe, "_get_recognition_quantiles"):
    word_meta = {
        ds_i.name: pd.DataFrame({"word_idx": np.arange(len(ds_i.word_onsets)),
                                 "recognition_quantile": pipe._get_recognition_quantiles(ds_i, pipe.params[0]).numpy()})
        for ds_i in store_ds.datasets
    }

store[f"epochs/predicted/{paradigm}/{run_name}"] = prepare_all_epochs({
    ds_i.name: pred_ts_i for ds_i, pred_ts_i in zip(store_ds.datasets, pred_ts)
}, word_meta=word_meta)
store[f"epochs/residual/{paradigm}/{run_name}"] = prepare_all_epochs({
    ds_i.name: ds_i.Y - pred_ts_i
    for ds_i, pred_ts_i in zip(store_ds.datasets, pred_ts)
}, word_meta=word_meta)

Predicting:   0%|          | 0/361 [00:00<?, ?it/s]

/net/vast-storage.ib.cluster/scratch/vast/cpl/jgauthie/scratch/bayesian-erp/berp/models/trf_em.py:1109: UserWarning: torch.searchsorted(): input value tensor is non-contiguous, this will lower the performance due to extra data copy when converting non-contiguous tensor to contiguous, please use contiguous input value tensor if possible. This message will only appear once per program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/aten/src/ATen/native/BucketizationUtils.h:33.)
  torch.bucketize(dataset.p_candidates[:, 0], self.recognition_quantile_edges_),


  0%|          | 0/361 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

  0%|          | 0/130 [00:00<?, ?it/s]

...

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/128 [00:00<?, ?it/s]

  0%|          | 0/154 [00:00<?, ?it/s]

/home/jgauthie/om2/miniconda/envs/berp/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'trf-cannon_3quantiles'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [16]:
store.flush()